In [7]:
from pompy_min import models, processors
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tqdm.notebook import tqdm
import sys

# Seed random number generator
seed = 15403997
rng = np.random.RandomState(seed)

# Define world model parameters
world_model_params = {
    'array_z': 0.,
    'n_x': 500,
    'n_y': 250,
    'puff_mol_amount': 8.3e8
}

# Define plume model parameters
plume_model_params = {
    'source_pos': (5., 0., 0.),
    'centre_rel_diff_scale': 2.,
    'puff_release_rate': 10,
    'puff_init_rad': 0.001**0.5,
    'puff_spread_rate': 0.001,
    'init_num_puffs': 10,
    'max_num_puffs': 1000,
    'model_z_disp': False,
}

# Define wind model parameters
wind_model_params = { 
    'n_x': 21,
    'n_y': 21,
    'u_av': 1.,
    'v_av': 0.,
    'k_x': 10.,
    'k_y': 10.,
    'noise_gain': 0.,
    'noise_damp': 0.0,
    'noise_bandwidth': 0.0,
    'use_original_noise_updates': False
}

# Define wind model simulation region
wind_region = models.Rectangle(x_min=0., x_max=100., y_min=-50., y_max=50.)

# Create wind model object
wind_model = models.WindModel(wind_region, rng=rng, **wind_model_params)

# Define plume simulation region
# This is a subset of the wind simulation region
sim_region = models.Rectangle(x_min=0., x_max=50., y_min=-12.5, y_max=12.5)

# Create plume model object
plume_model = models.PlumeModel(
    rng=rng, sim_region=sim_region, wind_model=wind_model, **plume_model_params)

# Create concentration array generator object
array_gen = processors.ConcentrationArrayGenerator(array_xy_region=sim_region, **world_model_params)

In [10]:
# Simulation timestep
dt = 0.01 #s
t = 6 #s

conc = np.zeros((int(t/dt),world_model_params['n_x'],world_model_params['n_y']))

conc[0,:,:] = array_gen.generate_single_array(plume_model.puff_array)

# Run wind model forward to equilibrate
for k in range(2000):
    wind_model.update(dt)


for i in tqdm(range(1,int(t/dt))):
    wind_model.update(dt)
    plume_model.update(dt)
    conc[i,:,:] = array_gen.generate_single_array(plume_model.puff_array)
    

  0%|          | 0/599 [00:00<?, ?it/s]

In [13]:
# convert time series to GIF using PIL
from PIL import Image

frames = []
maxx = np.max(conc)
for i in tqdm(range(conc.shape[0])):
    frame = Image.fromarray(np.uint8(conc[i]/maxx * 255).T)
    frames.append(frame)
frames[0].save('odor_plume.gif', format='GIF', append_images=frames[1:], save_all=True, fps=30, loop=0)

  0%|          | 0/600 [00:00<?, ?it/s]

In [15]:
    
# Set up figure
fig = plt.figure(figsize=(5, 2.5))
ax = fig.add_axes([0., 0., 1., 1.])
ax.axis('off')

# Display initial concentration field as image
conc_array = array_gen.generate_single_array(plume_model.puff_array)
conc_im = ax.imshow(conc_array.T, extent=(sim_region.x_min,sim_region.x_max,sim_region.y_min,sim_region.y_max),
                    vmin=0., vmax=1e10, cmap='Reds')

# Simulation timestep
dt = 0.01

# Run wind model forward to equilibrate
for k in range(2000):
    wind_model.update(dt)

# Define animation update function
def update(i):
    # Do 10 time steps per frame update
    for k in range(10):
        wind_model.update(dt)
        plume_model.update(dt)
    conc_array = array_gen.generate_single_array(plume_model.puff_array)
    conc_im.set_data(conc_array.T)
    return [conc_im]

# Animate plume concentration and save as MP4
anim = FuncAnimation(fig, update, frames=tqdm(range(400), file=sys.stdout), repeat=False)
anim.save('plume.gif',fps=30)

  0%|          | 0/400 [00:00<?, ?it/s]